Here's a breakdown of what's happening:

- **Import Libraries:** pandas is imported to manipulate data, and os to work with file names.

- **Excel Files List:** A list xlsx_files contains the names of the Excel files you wish to merge.

- **ExcelWriter Initialization:** pd.ExcelWriter('merged_file.xlsx') is used to create a new Excel file where all the data will be saved.

- **File Loop:** Loop through each .xlsx file in the list.

- **Read Excel File:** Each Excel file is read into a DataFrame using pd.read_excel().

- **Sheet Name:** Extract the file name without the extension to use as the sheet name using os.path.splitext().

- **Write to Excel:** Each DataFrame is written to the Excel writer object with its respective sheet name.

- **Save File:** Once the loop is complete, the Excel writer object saves the file, which will contain each original .xlsx file as a separate sheet.



In [1]:
import pandas as pd
import os

# Directory where the .xlsx files are located
input_directory = 'data/'

# Directory where the merged .xlsx file will be saved
output_directory = 'output/'

# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Output file name
output_file_name = 'output.xlsx'
output_file_path = os.path.join(output_directory, output_file_name)

# Get the list of .xlsx files you want to merge from the data directory
xlsx_files = [f for f in os.listdir(input_directory) if f.endswith('.xlsx')]

# Initialize a Pandas Excel writer using ExcelWriter class
with pd.ExcelWriter(output_file_path) as writer:

    for xlsx_file in xlsx_files:
        # Read each .xlsx file into a DataFrame
        df = pd.read_excel(os.path.join(input_directory, xlsx_file))

        # Get the name of the file without the extension
        sheet_name = os.path.splitext(xlsx_file)[0]

        # Write each DataFrame to the Excel writer object, specifying the sheet name
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# The merged Excel file will be saved in the output/ folder with separate sheets for each original file.

c:\Users\Hugo\miniconda3\envs\comprasgov\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Hugo\miniconda3\envs\comprasgov\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Hugo\miniconda3\envs\comprasgov\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Hugo\miniconda3\envs\comprasgov\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
# Importing Required Libraries
import pandas as pd
import numpy as np
import warnings

# Importing User Sensible Data
from sql.postgres_connection import (dbname, password, host, port, database)
from sql.comprasgov_siasg_dw_bps_extraction import (full_sql_query)

# Database Connection Function
def read_data_from_postgres(query):

    df = pd.DataFrame()  # assign a default value to df

    try:
        engine = create_engine(
            f'postgresql://{dbname}:{password}@{host}:{port}/{database}')
        df = pd.read_sql_query(query, engine)

    except Exception as e:
        print("An error occurred:", e)

    return df

# Test Connection
df = read_data_from_postgres(full_sql_query)
df_original = df.copy()
df

# df = df_original.copy()

An error occurred: name 'create_engine' is not defined


""
